# Get Data

여기에서는 주로 __Web Scraping__(crawling이라고도 함)을 다룬다. 

최근에는 대부분의 데이터가 web에 존재한다. data를 다운로드 해주는 친절한 사이트만 있는 것은 아니다. 수 많은 web page에서 원하는 데이터를 읽어와 사용해야 할 때가 있다. 

브라우저를 통해 일일이 페이지를 열어 원하는 자료를 긁어 저장한다면 막대한 시간이 들 뿐만 아니라 오류도 생겨나게 된다 

이럴 때 __Web Scraping__을 이용하여 원하는 데이터를 수집한다.

## REF


[Practical Introduction to Web Scraping in Python](https://realpython.com/python-web-scraping-practical-introduction/)

[Save and Load Machine Learning Models in Python with scikit-learn](https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/)


## Libs

In [1]:
from requests import get
from lxml import html
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import time

In [2]:
def simple_get(url):
    try:
        with closing( get( url, stream=True ) ) as resp:
            if is_good_response(resp): return resp.content
            else: return None
    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format( url, str(e) ) )
        return None

def is_good_response(resp):
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    print(e)

In [3]:
from bs4 import BeautifulSoup

def select_prescription_links( raw_html ):
    """
    처방 목록 페이지에서 각 처방 주소를 추출함
    """
    rst = []
    parsed_html = BeautifulSoup(raw_html, 'html.parser')
    for a in parsed_html.select('a'):
        if a.get('data-target') == 'prescription_view_win':
            rst.append(  a.get( 'href' ) )
    return rst

# 본초 추출
def select_medicine_ingredients( raw_html ):
    """
    각 처방 페이지에서 본초 구성 자료를 추출함
    """
    rst = []
    parsed_html = BeautifulSoup(raw_html, 'html.parser')
    for a in parsed_html.select('a'):
        if a.get('data-target') == 'medicine_view_win':
            rst.append(  a.text  )
    return rst

def select_medicine_symptoms( raw_html ):
    """
    각 처방 페이지에서 본초 구성 자료를 추출함
    """
    rst = []
    parsed_html = BeautifulSoup(raw_html, 'html.parser')
    for a in parsed_html.select('a'):
        if a.get('data-target') == 'disease_view_win':
            rst.append(  a.text  )
    return rst

## HTML pages

In [4]:
"""
특허청 한국전통지식포탈 > 전통의료 > 처방 
* 2019-02-20 기준
* 총 3,278건, 100건씩 33 페이지
* http://www.koreantk.com/ktkp2014/prescription/list-by-index.page?pageSize=100&pageNo=1
"""

n_pages = 33
url_base = "http://www.koreantk.com"
url_path = "/ktkp2014/prescription/list-by-index.page"
url_query = "?pageSize=100&pageNo={}"


## Scraping & Selecting

In [5]:
# Prepare Sandbox

temp_folder_path = "../data/_sandbox"

import os

if not os.path.exists( temp_folder_path ):
    os.makedirs( temp_folder_path )
    print( "Folder Created ... {}".format( temp_folder_path ) )

In [6]:
prescription_links = []

for i in range(n_pages)[:2]:
    url = url_base + url_path + url_query.format(i+1)
    raw_html = simple_get( url )
    with open( temp_folder_path + "/list_{:03d}.html".format(i), 'w', encoding="utf-8" ) as fl:
        fl.write( raw_html.decode('utf-8') )
    prescription_links += select_prescription_links( raw_html )
    print( "Web Scraping ... ", url )
    time.sleep( 5 )

print()
print( "# Prescription Link Lists")    
print( "\n".join( prescription_links[:10] ) )

Web Scraping ...  http://www.koreantk.com/ktkp2014/prescription/list-by-index.page?pageSize=100&pageNo=1
Web Scraping ...  http://www.koreantk.com/ktkp2014/prescription/list-by-index.page?pageSize=100&pageNo=2

# Prescription Link Lists
/ktkp2014/prescription/prescription-view.view?preCd=P0000001
/ktkp2014/prescription/prescription-view.view?preCd=P0020266
/ktkp2014/prescription/prescription-view.view?preCd=P0000002
/ktkp2014/prescription/prescription-view.view?preCd=P0008499
/ktkp2014/prescription/prescription-view.view?preCd=P0000003
/ktkp2014/prescription/prescription-view.view?preCd=P0004755
/ktkp2014/prescription/prescription-view.view?preCd=P0000004
/ktkp2014/prescription/prescription-view.view?preCd=P0020284
/ktkp2014/prescription/prescription-view.view?preCd=P0015600
/ktkp2014/prescription/prescription-view.view?preCd=P0015601


In [7]:
prescriptions = []
i = 0

for p_url_path in prescription_links[:10]:
    url = url_base + p_url_path
    raw_html = simple_get(url)
    i += 1
    with open( temp_folder_path + "/prescriptions_{:04d}.html".format(i), 'w', encoding="utf-8" ) as fl:
        fl.write( raw_html.decode('utf-8') )
    ingr = select_medicine_ingredients( raw_html )
    symp = select_medicine_symptoms( raw_html )
    prescriptions.append( { "herbs": ingr, "symptoms": symp } )
    print( "Web Scraping ... ", url )
    time.sleep( 5 )


Web Scraping ...  http://www.koreantk.com/ktkp2014/prescription/prescription-view.view?preCd=P0000001
Web Scraping ...  http://www.koreantk.com/ktkp2014/prescription/prescription-view.view?preCd=P0020266
Web Scraping ...  http://www.koreantk.com/ktkp2014/prescription/prescription-view.view?preCd=P0000002
Web Scraping ...  http://www.koreantk.com/ktkp2014/prescription/prescription-view.view?preCd=P0008499
Web Scraping ...  http://www.koreantk.com/ktkp2014/prescription/prescription-view.view?preCd=P0000003
Web Scraping ...  http://www.koreantk.com/ktkp2014/prescription/prescription-view.view?preCd=P0004755
Web Scraping ...  http://www.koreantk.com/ktkp2014/prescription/prescription-view.view?preCd=P0000004
Web Scraping ...  http://www.koreantk.com/ktkp2014/prescription/prescription-view.view?preCd=P0020284
Web Scraping ...  http://www.koreantk.com/ktkp2014/prescription/prescription-view.view?preCd=P0015600
Web Scraping ...  http://www.koreantk.com/ktkp2014/prescription/prescription-view.

## Save Data

### Separated Data Format ( CSV, TSV )

* [numpy.savetxt](https://docs.scipy.org/doc/numpy/reference/generated/numpy.savetxt.html)

In [8]:
output_path = temp_folder_path + "/prescriptions"

# prescription_herb.csv & prescription_symp.csv
separator = ","
csv_h = open( output_path + "_herb.csv", 'w', encoding="utf-8" )
csv_s = open( output_path + "_symp.csv", 'w', encoding="utf-8" )
for p in prescriptions:
    csv_h.write(   separator.join( p.get("herbs")      )  + "\n"  )
    csv_s.write(   separator.join( p.get("symptoms")   )  + "\n"  )
csv_h.close()
csv_s.close

# prescription_herb.tsf & prescription_symp.tsv
separator = "\t"
tsv_h = open( output_path + "_herb.tsv", 'w', encoding="utf-8" )
tsv_s = open( output_path + "_symp.tsv", 'w', encoding="utf-8" )
for p in prescriptions:
    tsv_h.write(   separator.join( p.get("herbs")     )  + "\n"  )
    tsv_s.write(   separator.join( p.get("symptoms")  )  + "\n"  )
tsv_h.close()
tsv_s.close()


### Structured Data Format ( JSON, YAML, XML )

* [json](https://docs.python.org/3/library/json.html)
* [PyYAML](https://pyyaml.org/wiki/PyYAMLDocumentation)

In [9]:
import json
import yaml

# prescriptions.json
json_str = json.dumps( prescriptions, ensure_ascii=False, sort_keys=True, indent=4 )
with open( "../data/kntk_formulas_toy.json", 'w', encoding="utf-8" ) as fl:
    fl.write( json_str )

# prescriptions.yaml
yaml_str = yaml.dump( prescriptions, allow_unicode=True, indent=4 )
with open( output_path + ".yaml", 'w', encoding="utf-8" ) as fl:
    fl.write( yaml_str )


### Binary Format for Python Object 

* [pickle](https://docs.python.org/3/library/pickle.html)
* [joblib](https://joblib.readthedocs.io/en/latest/)
* [numpy.save](https://docs.scipy.org/doc/numpy/reference/generated/numpy.save.html)

In [10]:
import pickle

# prescription.pickle
pickle.dump( prescriptions, open( output_path + ".pickle", 'wb' ) )